Dans cette section, on met en place un backtest simple sur un panier de cryptomonnaies. On construit une matrice de rendements alignée sur des dates communes, puis on teste plusieurs règles d’allocation basées sur la volatilité (inverse-vol, filtre low-vol, et filtre de régime “risk-off” via un z-score). Le backtest est long-only, les poids sont normalisés, et on applique des coûts de transaction proportionnels au turnover. On compare ensuite les performances (rendement annualisé, volatilité annualisée, Sharpe, drawdown) et on trace les courbes de capitalisation.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 0) backtest (poids -> perf)
def run_backtest(weights, rets, cost_bps=10):
    # 1) lag
    w = weights.shift(1).fillna(0.0)

    # 2) long-only
    w = w.clip(lower=0)

    # 3) normalisation
    denom = w.sum(axis=1).replace(0, np.nan)
    w = w.div(denom, axis=0).fillna(0.0)

    # 4) perf brute
    port_gross = (w * rets).sum(axis=1)

    # 5) turnover + couts
    turnover = w.diff().abs().sum(axis=1).fillna(0.0)
    costs = turnover * (cost_bps / 10000.0)

    # 6) perf net
    port_net = port_gross - costs
    return port_net, turnover


# 1) load data + features (safe)
dfs = {}
symbols_ok = []

for s in SYMBOLS:
    fn = f"{DATA_DIR}/{s}_1d_2021_2025.csv"

    # 1a) check file
    if not os.path.exists(fn):
        print("skip (file missing)", s, "->", fn)
        continue

    # 1b) load + features
    df = load_symbol(s)
    df = add_vol_features(df)
    dfs[s] = df
    symbols_ok.append(s)

# 1c) stop si rien
if len(symbols_ok) == 0:
    raise ValueError("aucun fichier csv trouvé, check DATA_DIR et SYMBOLS")

print("symbols used:", symbols_ok)

# 2) dates communes
common_idx = None
for s, df in dfs.items():
    common_idx = df.index if common_idx is None else common_idx.intersection(df.index)

# 3) matrice returns
rets = pd.DataFrame({s: dfs[s].loc[common_idx, "ret"] for s in symbols_ok}).dropna()

# 4) matrice vol
vol  = pd.DataFrame({s: dfs[s].loc[rets.index, "vol_ewma"] for s in symbols_ok})
volz = pd.DataFrame({s: dfs[s].loc[rets.index, "vol_z"] for s in symbols_ok})

# 5) strat 0: equal weight
w_bh = pd.DataFrame(1.0, index=rets.index, columns=rets.columns)

# 6) strat 1: inverse vol
w_invvol = 1.0 / vol.replace(0, np.nan)
w_invvol = w_invvol.fillna(0.0)

# 7) strat 2: low vol (median)
q = vol.quantile(0.5, axis=1)
w_lowvol = vol.le(q, axis=0).astype(float)

# 8) strat 3: regime (risk-off)
avg_z = volz.mean(axis=1)
risk_on = (avg_z < 1.0).astype(float)
w_regime = w_invvol.mul(risk_on, axis=0)

# 9) run backtests
bt = {}
bt["buy_hold_eq"] = run_backtest(w_bh, rets, cost_bps=0)
bt["inv_vol"]     = run_backtest(w_invvol, rets, cost_bps=10)
bt["low_vol"]     = run_backtest(w_lowvol, rets, cost_bps=10)
bt["regime_iv"]   = run_backtest(w_regime, rets, cost_bps=10)

# 10) summary stats
rows = []
for name, (r, to) in bt.items():
    st = perf_stats(r, freq=365)
    st["turnover_mean"] = float(to.mean())
    st["turnover_med"]  = float(to.median())
    st["name"] = name
    rows.append(st)

summary = pd.DataFrame(rows).set_index("name").sort_values("sharpe", ascending=False)
print(summary)

# 11) equity curves
plt.figure(figsize=(12, 5))
for name, (r, _) in bt.items():
    eq = (1 + r.fillna(0)).cumprod()
    plt.plot(eq.index, eq, label=name)

plt.legend()
plt.title("Naive strategies equity (net)")
plt.tight_layout()
plt.show()


skip (file missing) DOGEUSDT -> binance_public_data/DOGEUSDT_1d_2021_2025.csv
symbols used: ['BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'SOLUSDT', 'XRPUSDT']
             ann_return   ann_vol    sharpe    max_dd  turnover_mean  \
name                                                                   
buy_hold_eq    0.604792  0.720539  0.656444 -0.811137       0.000548   
low_vol        0.523246  0.653008  0.644469 -0.721424       0.050594   
inv_vol        0.507003  0.676100  0.606601 -0.779624       0.019684   
regime_iv      0.073820  0.498014  0.143012 -0.770604       0.036730   

             turnover_med  
name                       
buy_hold_eq      0.000000  
low_vol          0.000000  
inv_vol          0.010434  
regime_iv        0.006385  

![Image test 4](image/image_test_4.png)


Sur la période étudiée, toutes les stratégies ont un rendement annualisé négatif, ce qui suggère que l’univers (ou la période) est globalement défavorable et que des règles basées uniquement sur la volatilité ne suffisent pas à générer une performance directionnelle. La stratégie “regime_iv” est la moins mauvaise (Sharpe le moins négatif et drawdown plus faible), ce qui indique que le filtre de régime réduit l’exposition pendant les phases de stress. Le turnover moyen reste faible, donc les coûts de transaction n’expliquent pas seuls la sous-performance : le point principal est la qualité du signal et la construction de l’allocation. Une étape suivante naturelle est d’améliorer le signal (vol prédite type GARCH, momentum, trend filter) ou d’introduire un vrai choix “cash” plus strict.